# Seperating stock to forcast for certain dates depending on the product

N-BEATS only works on large datasets, with this in mind, it might only be possible to perform a nbeats model on the full dataset, not seperating the data by customer.

In [1]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn darts 


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
import sys
sys.path.append('../../src')

In [4]:
from data_config import ProductHandler as ph

/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:13: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:15: DtypeWarning: Columns (5,29,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:18: DtypeWarning: Columns (43,57,106,141,146,211,212,247,255,294,298,316,327,332,350,354) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  sp_inv_adds = pd.read_csv(


----- DROP BOOLEAN AND NULL COLUMNS (NOT NEEDED) -----

Merged Data - Columns to be dropped (Missing Vals): ['BackStage_DefaultReportForm', 'BackStage_SmartMarkSet', 'BillCounty', 'CoreType', 'EndUserNum', 'EndUserPO', 'Equip3_ID', 'Equip3_Status', 'Est_v_Act_Notes', 'Ink_Status', 'JDF_Note_to_DFE', 'JDF_Send_Msg', 'MiscChargeDesc1', 'MiscChargeDesc2', 'MiscChargeDesc3', 'MiscChargeDesc4', 'PlateStat', 'ProofStat', 'Roto_CEL_Product_ID', 'Roto_Quote_Line_ID', 'Roto_Quote_Number', 'Schedule_Status', 'ShipAttn_EmailAddress', 'ShipCounty', 'ShipStat', 'Tag_x', 'Tool3Descr', 'Tool4Descr', 'Tool5Descr', 'ToolNo3', 'ToolNo4', 'ToolNo5', 'ToolStat', 'UserDef_MR_1_Lb', 'UserDef_MR_2_Lb', 'Assigned', 'Desc2_x', 'ediLineNumber', 'eTraxx_Customer_Notes', 'Location_x', 'PO_Number', 'Adhesive', 'Alternate', 'eTraxx_Forecast_Range', 'Material', 'Tag_y', 'UPC']

Merged Data - Columns to be dropped (All cols with rows same vals): ['Act_OTHER_Hours', 'ActArtwork', 'ActFinMaterial', 'ActPostPressHours',

/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(
/Users/mwilko777/Desktop/Project/Software Artifact/notebooks/n_beat/../../src/datasets.py:110: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expe

In [5]:
ph.custom_prod_set()

All custom codes for existing products: 
{'AZU', '42W', '23P', 'SQU', 'LOC', 'PJ-', '11U', '337', '09V', '477', 'SHF', '45Y', '07O', '144', '14T', '866', '35Z', '544', '30U', '285', '21Y', '885', '372', '185', '11X', '666', '629', '912', '120', 'STB', '134', '90Z', '10U', 'ARO', '240', '830', '12S', '162', '172', 'UG-', '0Z-', 'OO-', 'RAN', '664', 'A-T', 'D-0', '260', 'TFO', '194', '073', '388', 'BLO', '66Z', '-11', '279', '462', '549', '864', 'LG-', 'SF-', '52X', 'DSF', 'OWL', 'P-0', '636', '587', '12R', '26W', '34V', '373', 'REI', '07V', '160', '326', '60S', '690', '794', '215', '03X', '059', '97X', '07S', '71Z', '40Y', 'JBM', 'Y-T', '181', 'GPB', '64V', '930', '479', '771', 'GC-', '022', '470', '262', '287', 'A-0', '161', '463', 'C-T', '076', 'BJ-', '28Y', 'AN-', '88Z', '60U', '60W', '5X-', 'OG-', '877', '193', '02T', '697', '41U', '947', '472', '73T', '05V', '72X', '671', '782', '176', '20Z', '481', '76T', '396', 'NTF', '735', '238', '543', '589', '527', '232', '55Y', '15Y', '368',

In [6]:
# define the features and target variable from 'product_sales'
# CHANGE THE CUSTOMER CODE HERE >>>>>>>>>>
custom_code_df = pd.DataFrame(ph.get_custom_code_data('ALB'))

# customer code for later referencing
custom_ref = custom_code_df.loc[0, 'ProductNumber'][:3].lower().upper()
print(custom_ref)

custom_code_df.head()

ALB


,ProductNumber,order_year,order_month,order_week,order_weekday,is_weekend,OrderDate,Customer_Num,OrderQuantity,prev_month_sales,...,moving_avg_3m,moving_avg_6m,moving_avg_12m,moving_avg_18m,sales_2022,sales_2023,sales_2024,growth_2023,growth_2024,PhysicalInv
0,ALB-005-0022Y,2024,9,39,0,0,2024-09-23,191.0,160000,110031.53337,...,160000.000000,160000.000000,160000.000000,160000.000000,0,0,160000,NaN,inf,8000
1,ALB-005-0022Z,2024,7,30,1,0,2024-07-23,191.0,96000,110031.53337,...,96000.000000,96000.000000,96000.000000,96000.000000,0,0,96000,NaN,inf,0
2,ALB-005-0023Y,2024,11,45,1,0,2024-11-05,191.0,72000,88000.00000,...,57333.333333,57333.333333,57333.333333,57333.333333,0,0,72000,NaN,inf,8000
3,ALB-005-0023Y,2024,12,1,0,0,2024-12-30,191.0,12000,110031.53337,...,12000.000000,12000.000000,12000.000000,12000.000000,0,0,12000,NaN,inf,8000
4,ALB-005-0023Y,2024,12,49,3,0,2024-12-05,191.0,88000,12000.00000,...,50000.000000,50000.000000,50000.000000,50000.000000,0,0,88000,NaN,inf,8000


In [7]:
from app_utils import Transform as trans
from model_utils import *
from sklearn.pipeline import Pipeline

# Main imports for N-BEAT model
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [17]:
# Compute OrderQuantity values with a zscore with threshold of 3 and remove them (Outlier removal)
custom_code_df = trans.compute_zscore(custom_code_df)

# Ensure datetime col is formatted properly since model is a time series
custom_code_df["OrderDate"] = pd.to_datetime(custom_code_df["OrderDate"])
custom_code_df.head()

custom_code_df.count()

ProductNumber         2850
order_year            2850
order_month           2850
order_week            2850
order_weekday         2850
is_weekend            2850
OrderDate             2850
Customer_Num          2850
OrderQuantity         2850
prev_month_sales      2850
prev_week_sales       2850
prev_2_month_sales    2850
prev_3_month_sales    2850
var_3m                2850
var_6m                2850
var_12m               2850
var_18m               2850
log_var_3m            2544
log_var_6m            2544
log_var_12m           2544
log_var_18m           2544
moving_avg_3m         2850
moving_avg_6m         2850
moving_avg_12m        2850
moving_avg_18m        2850
sales_2022            2850
sales_2023            2850
sales_2024            2850
growth_2024           2564
PhysicalInv           2850
z_score               2850
dtype: int64

In [9]:
# Drop columns with fewer than 2500 non-null values
custom_code_df = custom_code_df.loc[:, custom_code_df.count() >= 2500]

In [10]:
X = custom_code_df.drop('OrderQuantity', axis=1)
# Set y explicitly as the target column name.
y = "OrderQuantity"
# y = custom_code_df.OrderQuantity

In [11]:
# Set y as the target column name explicitly
y = "OrderQuantity"

# Ensure that "ProductNumber" is only a column, not also in the index
if "ProductNumber" in custom_code_df.index.names:
    custom_code_df = custom_code_df.reset_index(drop=True)

# Reset index cleanly if needed
custom_code_df = custom_code_df.reset_index(drop=True)

# Exclude non-numeric columns from covariates (including OrderDate)
X = [col for col in custom_code_df.columns if col not in ['ProductNumber', 'OrderQuantity', 'OrderDate']]

series_list = []
covariates_list = []

# Group by ProductNumber
for product_id, group in custom_code_df.groupby("ProductNumber"):
    # Ensure 'OrderDate' is a datetime and sorted
    group["OrderDate"] = pd.to_datetime(group["OrderDate"])
    group = group.sort_values("OrderDate").set_index("OrderDate")
    
    # Reindex to weekly frequency, filling missing dates with forward-fill (or use fill_value=0)
    group = group.asfreq("W", method="ffill")
    
    # Reset index to make 'OrderDate' a column again
    group_reset = group.reset_index()
    
    # Replace infinite values and fill NaNs in the group
    group_reset = group_reset.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Target series (OrderQuantity over time)
    target_series = TimeSeries.from_dataframe(
        group_reset,
        time_col="OrderDate",
        value_cols=[y],
        freq="W",
        fill_missing_dates=True
    )

    # Skip groups that yield an empty TimeSeries
    if len(target_series) == 0:
        print(f"Skipping ProductNumber {product_id} due to empty target series.")
        continue

    # Covariates (only numeric columns; ensure 'OrderDate' is excluded)
    covariates = TimeSeries.from_dataframe(
        group_reset,
        time_col="OrderDate",
        value_cols=X,
        freq="W",
        fill_missing_dates=True
    )

    series_list.append(target_series)
    covariates_list.append(covariates)

# Check if we have any valid series before scaling
if len(series_list) == 0:
    raise ValueError("No valid time series were generated. Check your data and grouping.")

# Scale the data (critical for N-BEATS)
target_scaler = Scaler()
scaled_series = target_scaler.fit_transform(series_list)

covariate_scaler = Scaler()
scaled_covariates = covariate_scaler.fit_transform(covariates_list)


Skipping ProductNumber ALB-005-0038W due to empty target series.
Skipping ProductNumber ALB-005-0066Y due to empty target series.


In [12]:
# Split each product's series into train/val
train_series, val_series = [], []
train_covariates, val_covariates = [], []

for series, covariate in zip(scaled_series, scaled_covariates):
    train, val = series.split_before(0.8)  # 80% training, 20% validation
    train_cov, val_cov = covariate.split_before(0.8)
    
    train_series.append(train)
    val_series.append(val)
    train_covariates.append(train_cov)
    val_covariates.append(val_cov)

In [13]:
# Check if train_series is a Darts TimeSeries object or a list
if hasattr(train_series, 'time_index'):
    # Print concise info for Darts TimeSeries object
    print(f"Time series from {train_series.time_index[0]} to {train_series.time_index[-1]}")
    print(f"Length: {len(train_series)}")
else:
    # If it's a list of time series
    print(f"Total number of time series: {len(train_series)}")
    for i, ts in enumerate(train_series):
        if hasattr(ts, 'time_index'):  # Darts TimeSeries
            print(f"Series {i} length: {len(ts)}")
        else:  # Regular list
            print(f"Series {i} (list) length: {len(ts)}")

# Check if train_covariates is a Darts TimeSeries object or a list
if hasattr(train_covariates, 'time_index'):
    # Print concise info for Darts TimeSeries object
    print(f"Covariates from {train_covariates.time_index[0]} to {train_covariates.time_index[-1]}")
    print(f"Length: {len(train_covariates)}")
else:
    # If it's a list of covariates
    print(f"Total number of covariates: {len(train_covariates)}")
    for i, cov in enumerate(train_covariates):
        if hasattr(cov, 'time_index'):  # Darts TimeSeries
            print(f"Covariate {i} length: {len(cov)}")
        else:  # Regular list
            print(f"Covariate {i} (list) length: {len(cov)}")


Total number of time series: 313
Series 0 length: 5
Series 1 length: 10
Series 2 length: 0
Series 3 length: 4
Series 4 length: 9
Series 5 length: 6
Series 6 length: 6
Series 7 length: 3
Series 8 length: 9
Series 9 length: 7
Series 10 length: 3
Series 11 length: 6
Series 12 length: 4
Series 13 length: 4
Series 14 length: 6
Series 15 length: 6
Series 16 length: 6
Series 17 length: 6
Series 18 length: 6
Series 19 length: 10
Series 20 length: 9
Series 21 length: 4
Series 22 length: 10
Series 23 length: 4
Series 24 length: 9
Series 25 length: 6
Series 26 length: 5
Series 27 length: 9
Series 28 length: 10
Series 29 length: 3
Series 30 length: 9
Series 31 length: 6
Series 32 length: 4
Series 33 length: 6
Series 34 length: 0
Series 35 length: 4
Series 36 length: 6
Series 37 length: 6
Series 38 length: 9
Series 39 length: 4
Series 40 length: 9
Series 41 length: 4
Series 42 length: 10
Series 43 length: 3
Series 44 length: 9
Series 45 length: 4
Series 46 length: 4
Series 47 length: 4
Series 48 le

In [14]:
# If is a pandas DataFrame with columns representing time series
series_lengths = custom_code_df.apply(lambda col: len(col.dropna()))
print(series_lengths)


ProductNumber         2850
order_year            2850
order_month           2850
order_week            2850
order_weekday         2850
is_weekend            2850
OrderDate             2850
Customer_Num          2850
OrderQuantity         2850
prev_month_sales      2850
prev_week_sales       2850
prev_2_month_sales    2850
prev_3_month_sales    2850
var_3m                2850
var_6m                2850
var_12m               2850
var_18m               2850
log_var_3m            2544
log_var_6m            2544
log_var_12m           2544
log_var_18m           2544
moving_avg_3m         2850
moving_avg_6m         2850
moving_avg_12m        2850
moving_avg_18m        2850
sales_2022            2850
sales_2023            2850
sales_2024            2850
growth_2024           2564
PhysicalInv           2850
z_score               2850
dtype: int64


In [15]:
# Find best params for NBEATS, once found comment out due to time and compute resources
parameter_grid = param_grids(NBEATSModel.__name__)
nbeat_params = find_best_hyperparameters(NBEATSModel, parameter_grid, train_series, train_series)

ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The 

Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'batch_size': 32, 'n_epochs': 50, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'batch_size': 32, 'n_epochs': 100, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'batch

ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The 

Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 4, 'layer_widths': 256, 'dropout': 0.1, 'optimizer_kwargs': {'lr': 0.0001}, 'batch_size': 64, 'n_epochs': 50, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 4, 'layer_widths': 256, 'dropout': 0.1, 'optimizer_kwargs': {'lr': 0.0001}, 'batch_size': 64, 'n_epochs': 100, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 5, 'num_blocks': 3, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'bat

ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The 

...

Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'batch_size': 64, 'n_epochs': 50, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.001}, 'batch_size': 64, 'n_epochs': 100, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.0001}, 'b

ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The 

Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.1, 'optimizer_kwargs': {'lr': 0.001}, 'batch_size': 64, 'n_epochs': 100, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.1, 'optimizer_kwargs': {'lr': 0.0001}, 'batch_size': 32, 'n_epochs': 50, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 256, 'dropout': 0.1, 'optimizer_kwargs': {'lr': 0.0001}, '

ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
ValueError: The 

Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 3, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.0001}, 'batch_size': 32, 'n_epochs': 100, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 3, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.0001}, 'batch_size': 64, 'n_epochs': 50, 'random_state': 42}: The dataset contains some time series that are too short to contain `max(self.input_chunk_length, self.shift + self.output_chunk_length)` (0-th series)
Error with params {'input_chunk_length': 6, 'output_chunk_length': 1, 'num_stacks': 10, 'num_blocks': 3, 'num_layers': 2, 'layer_widths': 128, 'dropout': 0.0, 'optimizer_kwargs': {'lr': 0.0001}, 

In [16]:
nbeat_tuned = NBEATSModel(**nbeat_params)

TypeError: darts.models.forecasting.nbeats.NBEATSModel() argument after ** must be a mapping, not NoneType

In [ ]:
nbeat_tuned.fit(
    series=train_series,
    past_covariates=train_covariates, # Features known in the past (e.g., prev_month_sales)
    val_series=val_series,
    val_past_covariates=val_covariates,
    verbose=True
)

In [ ]:
# Forecast for the first product
product_idx = 0
forecast = nbeat_tuned.predict(
    n=len(val_series[product_idx]),  # Match validation horizon
    series=train_series[product_idx],
    past_covariates=train_covariates[product_idx]
)

# Inverse scaling
forecast_unscaled = target_scaler.inverse_transform(forecast)
actual_unscaled = target_scaler.inverse_transform(val_series[product_idx])

# Plot
train_series[product_idx].plot(label="Train")
actual_unscaled.plot(label="Actual")
forecast_unscaled.plot(label="Forecast")
plt.legend()
plt.show()

In [ ]:
# Define forecast horizon (e.g., predict next 6 months)
HORIZON = 6  # Should match `output_chunk_length` (defined in model params)

print('----------- TRAINING METRICS -----------')
train_metrics = evaluate_timeseries(
    model=nbeat_tuned,
    val_series=train_series, # Training data (TimeSeries objects)
    val_covariates=train_covariates, # Training features (TimeSeries)
    horizon=HORIZON,
    target_scaler=target_scaler
)

print('\n\n----------- TESTING METRICS -----------')
test_metrics = evaluate_timeseries(
    model=nbeat_tuned,
    val_series=val_series, # Validation data (TimeSeries)
    val_covariates=val_covariates, # Validation features (TimeSeries)
    horizon=HORIZON,
    target_scaler=target_scaler
)